# ROUGH DRAFT 1

# Capstone Project - Battle of the Neighborhoods
### Applied Data Science Capstone - IBM Coursera

### Table of Contents

* [1. Introduction: Business Problem](#introduction)
* [2. Data Description](#data)
* [3. Methodology](#methodology)
* [4. Analysis](#analysis)
* [5. Results and Discussion](#results)
* [6. Conclusion](#conclusion)

<a id="introduction"></a>

### 1. Introduction: Business Problem - Opening an Argentinean Restaurant or Cafe

This project will focus around two major cities. Cities known for diversity. With diversity, brings diverse resturaunts, cafes and more, which bring people together. If you're a stakeholder, entrepreneur, or chef interested in opening an Argentinean resturaunt or cafe in New York city or Toronto, this will be for you. 

Restaurant industry is huge in both of these cities, so we will try to identify the best location. To find the best location, we will use a number of data points such as: popular visiting locations, population percentage per borough, amount of current vertical competition, and no argentinean restaurants in the area. 

Using efficient data science tools, we will generate two promisions locations for both major cities. Each location will have the benefits explained and can be decided by the stakeholder.


<a id="data"></a>

### 2. Data Description

The expected data to be applied for our decision are: 
* Top five venues visited per neighborgood(borough)
* Population percentage per borough
* Amount of competition (Restaurant & Cafes) per neighborhood
* Current location and foursquare information for Argentinean Resturaunts

We will attempt to seperate neighborhoods using Choropleth maps and/or by radius. 

The following data sources will be utilized to extract and generate the required information. The list below represents the order in which the data will be generated. 

* New York City Census Information. CSV.
* Canada Census 2016: Population & Density information
* Wiki: Toronto Postal Codes
* Foursquare API: Retreive results for amount of restaurants and cafes, check in information,  popularity to provide competition insight.
* Google Maps API Geocode: Product location information, visualize locations for comparision, provide results.

In [1]:
########################################
#### Libraries used in this section ####
########################################

import pandas as pd # library for data analsysis
import requests
from bs4 import BeautifulSoup
import config
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranforming json file into a pandas dataframe library
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import folium # plotting library

#### 2a. New York City Census Information

New York City Census Information will be used to gather the population data for each county located in New York City. County population data will help determine which area we select for a future resturaunt location. 

In [2]:
# Import: NYC Census Data CSV
links={'pop':'C:/Users/pompa/gitProjects/Coursera_Capstone/Coursera_Capstone/QuickFactsFeb2020.csv'}
# Create: DataFrame with Census Data
df_POP = pd.read_csv(links["pop"]) 

In [3]:
# Verify Data: Identify what needs to be removed.
df_POP.head()

,Fact,Fact Note,"New York city, New York","Value Note for New York city, New York","Bronx County (Bronx Borough), New York","Value Note for Bronx County (Bronx Borough), New York","Kings County (Brooklyn Borough), New York","Value Note for Kings County (Brooklyn Borough), New York","New York County (Manhattan Borough), New York","Value Note for New York County (Manhattan Borough), New York","Queens County (Queens Borough), New York","Value Note for Queens County (Queens Borough), New York","Richmond County (Staten Island Borough), New York","Value Note for Richmond County (Staten Island Borough), New York"
0,"Population estimates, July 1, 2019, (V2019)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Population estimates, July 1, 2018, (V2018)",NaN,"8,398,748",NaN,"1,432,132",NaN,"2,582,830",NaN,"1,628,701",NaN,"2,278,906",NaN,"476,179",NaN
2,"Population estimates base, April 1, 2010, (V2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Population estimates base, April 1, 2010, (V2...",NaN,"8,174,988",NaN,"1,384,603",NaN,"2,504,717",NaN,"1,586,360",NaN,"2,230,578",NaN,"468,730",NaN
4,"Population, percent change - April 1, 2010 (es...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Clean Data: Drop NaN columns not needed.
df_POP.drop("Fact Note", axis = 1, inplace=True)
df_POP.drop("Value Note for New York city, New York", axis = 1, inplace=True)
df_POP.drop("Value Note for Bronx County (Bronx Borough), New York", axis = 1, inplace=True)
df_POP.drop("Value Note for Kings County (Brooklyn Borough), New York", axis = 1, inplace=True)
df_POP.drop("Value Note for New York County (Manhattan Borough), New York", axis = 1, inplace=True)
df_POP.drop("Value Note for Queens County (Queens Borough), New York", axis = 1, inplace=True)
df_POP.drop("Value Note for Richmond County (Staten Island Borough), New York", axis = 1, inplace=True)
df_POP.drop([2, 4], axis = 0, inplace=True)

In [5]:
# Clean Data: Drop extra row
df_POP.drop(0, axis = 0, inplace=True)
df_POP.head()

,Fact,"New York city, New York","Bronx County (Bronx Borough), New York","Kings County (Brooklyn Borough), New York","New York County (Manhattan Borough), New York","Queens County (Queens Borough), New York","Richmond County (Staten Island Borough), New York"
1,"Population estimates, July 1, 2018, (V2018)","8,398,748","1,432,132","2,582,830","1,628,701","2,278,906","476,179"
3,"Population estimates base, April 1, 2010, (V2...","8,174,988","1,384,603","2,504,717","1,586,360","2,230,578","468,730"
5,"Population, percent change - April 1, 2010 (es...",2.7%,3.4%,3.1%,2.7%,2.2%,1.6%
6,"Population, Census, April 1, 2010","8,175,133","1,385,108","2,504,700","1,585,873","2,230,722","468,730"
7,"Persons under 5 years, percent",6.5%,7.2%,7.2%,4.7%,6.2%,5.8%


In [6]:
# Clean Data: Rename Columns, drop data.
df_POP.columns = ['2018', 'New York city', 'Bronx County', 'Kings County', 'New York County', 'Queens County', 'Richmond County']
df_POP2 = df_POP
df_POP2 = df_POP2[:1]
df_POP2.drop("2018", axis = 1, inplace=True)

C:\Users\pompa\Anaconda3\envs\ipykernel_py2\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


#### 2b. New York City population & population per county.

In [7]:
# Second df with only population
df_POP2

,New York city,Bronx County,Kings County,New York County,Queens County,Richmond County
1,"8,398,748","1,432,132","2,582,830","1,628,701","2,278,906","476,179"


In [8]:
# Filtered: pop by county in NYC.
nyc_Totalpop = df_POP2['New York city'].str.replace(',', '').astype(int)
bronxCounty = df_POP2['Bronx County'].str.replace(',', '').astype(int)
kingsCounty = df_POP2['Kings County'].str.replace(',', '').astype(int)
nyCounty = df_POP2['New York County'].str.replace(',', '').astype(int)
queensCounty = df_POP2['Queens County'].str.replace(',', '').astype(int)
richmondCounty = df_POP2['Richmond County'].str.replace(',', '').astype(int)

In [9]:
# Filtered: Percentage of population per County
bronxCounty_popPercentage = int(bronxCounty) / int(nyc_Totalpop)
kingsCounty_popPercentage = int(kingsCounty) /  int(nyc_Totalpop)
nyCounty_popPercentage = int(nyCounty) / int(nyc_Totalpop)
queensCounty_popPercentage = int(queensCounty) / int(nyc_Totalpop)
richmondCounty_popPercentage = int(richmondCounty) / int(nyc_Totalpop)

In [10]:
# Clean: Initialise data to lists. 
data = [{'Bronx County': bronxCounty_popPercentage, 'Kings County': kingsCounty_popPercentage, 'New York County': nyCounty_popPercentage, 'Queens County': queensCounty_popPercentage, 'Richmond County': richmondCounty_popPercentage}] 

# Creates Percentage of population per County DataFrame. 
df_popPercentage = pd.DataFrame(data)
df_popPercentage

,Bronx County,Kings County,New York County,Queens County,Richmond County
0,0.170517,0.307526,0.193922,0.271339,0.056696


#### 2c. Latitude/Longitude for NYC counties.

In [11]:
### Lat Lng of Boroughs in New York City
# Filter: Used 'address'
address = 'Bronx County, New York City, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.85048545 -73.84040355802088


In [12]:
# Create DataFrame with Lat/Lng for each borough in NYC.
latlng_nyc ={'Bronx County':['40.85048545', '-73.84040355802088'], 'Kings County':['40.64530975', '-73.9550230275334'], 'New York County':['40.7810178', '-73.959299675'], 'Queens County':['40.652492699999996', '-73.7914214158161'], 'Richmond County':['40.564209', '-74.12530461995391']} 
df_NYCB_latlng = pd.DataFrame(latlng_nyc, index =['latitude', 'longitude']) 
df_NYCB_latlng = df_NYCB_latlng.astype('float')
df_NYCB_latlng

,Bronx County,Kings County,New York County,Queens County,Richmond County
latitude,40.850485,40.645310,40.781018,40.652493,40.564209
longitude,-73.840404,-73.955023,-73.959300,-73.791421,-74.125305


#### 2d. Search for a Neighborhood in Toronto

In [20]:
# Import Wiki table information.
weblink = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(weblink.content,'lxml')
table = soup.find_all('tbody')[0]

# Select Table with data
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_rows = table.find_all('tr')

### Data for table
data = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    data.append(row)
#print(data)
#print(df[0].to_json(orient='records'))

### Convert imported data to pandas DataFrame
df = pd.DataFrame(data, columns=["Postcode", "Borough", "Neighbourhood"])

#### Clean Data

In [21]:
# Remove \n
df = df.replace(r'\n','', regex=True)
df.columns = df.columns.str.strip()
# verify column headers
#df.columns

# verify column headers
df.columns

# Group columns by postal code and borough, removing duplicate postcode value
# and moving the duplicate data in borough to Neighbourhood.
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

# Rename headers
df.rename(columns ={'Postcode': 'PostalCode'}, inplace=True)
df.rename(columns ={'Neighbourhood': 'Neighborhood'}, inplace=True)

# Verify neighbourhood has values in all rows.
df.isin(['Not Available']).any().any()

False

#### Generating Lat & Lng by Postal Code.

In [22]:
# Main list to run
postal_code = df['PostalCode'].tolist()

In [23]:
# Main list
list1 = postal_code
# Getting length of list 
api_key = config.api_key
i = 0
# Iterating using while loop 
while i < len(list1):  
    element = list1[i]
    geoCodeUrl = "https://maps.googleapis.com/maps/api/geocode/json?components=postal_code:{}|country:CA&key={}".format(
        element,
        api_key) 
    
    # make the GET request
    lookup = requests.get(geoCodeUrl)
    data = lookup.json()  
    
    # If no results are found for postal code, skip and move on.
    if (data['status'] == 'ZERO_RESULTS'):
        i += 1
        continue
    # Instantiate an empty dict
    latlng = {}
    # latlng information
    latlng['PostalCode'] = data['results'][0]['address_components'][0]['long_name']
    latlng['Longname'] = data['results'][0]['address_components'][1]['long_name']
    latlng['Latitude'] = data['results'][0]['geometry']['location']['lat']
    latlng['Longitude'] = data['results'][0]['geometry']['location']['lng']

    with open('data.json', 'r') as j:
        json_data = json.load(j)
    # convert data to list if not
        if type(json_data) is dict:
            json_data = [json_data]

    # use append() to add to list
    json_data.append(latlng)    

    #write list to file
    with open('data.json', 'w') as outfile:
        json.dump(json_data, outfile)
    # Normalize data. Flatten JSON.
    data_normalized = pd.json_normalize(json_data) # flatten JSON
    i += 1 
       
    if i == len(list1):
        break

print(data_normalized)

    PostalCode     Longname   Latitude  Longitude venues
0          M1J  Scarborough  43.744734 -79.239476    NaN
1          M1B  Scarborough  43.806686 -79.194353    NaN
2          M1C  Scarborough  43.784535 -79.160497    NaN
3          M1E  Scarborough  43.763573 -79.188711    NaN
4          M1G  Scarborough  43.770992 -79.216917    NaN
..         ...          ...        ...        ...    ...
234        M9N      Toronto  43.706876 -79.518188    NaN
235        M9P    Etobicoke  43.696319 -79.532242    NaN
236        M9R    Etobicoke  43.688905 -79.554724    NaN
237        M9V    Etobicoke  43.739416 -79.588437    NaN
238        M9W    Etobicoke  43.706748 -79.594054    NaN

[239 rows x 5 columns]


In [24]:
# Open new json file created above.
with open('data.json', 'r') as j:
    json_data = json.load(j)

In [25]:
# Rename longname to borough on new generated dataframe
data_df = pd.DataFrame(json_data)
data_df.rename(columns = {'Longname':'Borough'}, inplace=True)
data_df.head()

,PostalCode,Borough,Latitude,Longitude,venues
0,M1J,Scarborough,43.744734,-79.239476,NaN
1,M1B,Scarborough,43.806686,-79.194353,NaN
2,M1C,Scarborough,43.784535,-79.160497,NaN
3,M1E,Scarborough,43.763573,-79.188711,NaN
4,M1G,Scarborough,43.770992,-79.216917,NaN


In [26]:
####################################################################
#### Combine Dataframes: Postal Code, Neighborhood, with Latlng ####
####################################################################

In [27]:
# Drop duplicate rows
# Useful if while loop is ran multiple times.
df = df.drop_duplicates()
data_df = data_df.drop_duplicates()

# Merge df and df_data
# Postal code with 
df_merge_col = pd.merge(df, data_df, on='PostalCode')

# Clean df_merge_col
df_merge_col.rename(columns = {'Borough_x':'Borough'}, inplace=True)
df_merge_col.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace=True)
df_merge_col = df_merge_col.drop(['Borough_y'], axis=1)
df_merge_col.head()

TypeError: unhashable type: 'list'

<a id="methodology"></a>

### 3. Methodology

## FIX THIS. Not complete. needs to be in order.

The goal is to provide two possible locations for both New York City and Toronto, to build a restaurant and/or cafe. By focusing on high population density areas, popular areas for food, and no current Argentinean restaurants around.

First, we collect data from our sources such as: Neighborhood latitude and longitude information, number of resturaunts and cafes in both major cities, location and number of Argentinean resturaunts in both cities, and competition information. 

Second, we calculate the population density per neighborhood and create a heatmap showing density per neighborhood. We will also use heatmaps to distingush the amount of resturaunts and cafes per neighborgood, per city. 

Third, we will create clusters of neighborhoods which meet our requirements in providing promising locations to discuss with stakeholders. Using K-Means clustering, we will present a a map with the following locations found. 

<a id="analysis"></a>

### 4. Analysis

Analysis will combine the data acquired in section #2, combine Google location data with Four Square information to find the solutions needed. Solutions include: Argentinian restauraunts per county in NYC and Toronto, competition in each area, and population percentage per county.

Using the location data acquired in section #2, lets use the Latitude, Longitude, and County information to <b>locate the number of Argentinian and Empanada restauraunts by NYC County.</b>

In [41]:
# Fourquare Information. 
VERSION = '20180604'
LIMIT = 30
radius = 6000
search_query = ['Empanadas', 'Argentinian', 'chimichurri']
CLIENT_ID = config.CLIENT_ID
CLIENT_SECRET = config.CLIENT_SECRET

# Bronx lat/lng

#### 4a. Bronx County Data - Restaurant

In [42]:
########################################
### Retreive Four Square Information ###
########################################

#Bronx County Results for 'Argentinean' locations
# Bronx lat/lng
bronx_lat = float(df_NYCB_latlng['Bronx County'].latitude)
bronx_lng = float(df_NYCB_latlng['Bronx County'].longitude)
lat = bronx_lat = df_NYCB_latlng['Bronx County']['latitude']
lng = bronx_lng = df_NYCB_latlng['Bronx County']['longitude']

In [43]:
# Create: Generate empty DataFrame to populate.
dataframe = pd.DataFrame()

In [44]:
# Data: Loop through a number of search queries. We call the Four Square API to search 
# and query specific keywords related to Argentinian restauraunts.
# We will use this loop for each county to find data by specific keywords. 
# 'dataframe' is populated on every loop and will provide results of query per location requested.

search_query = ['chimichurri', 'empanadas', 'Argentinian']
i = 0
while i < len(search_query):
    element = search_query[i]
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, element, radius, LIMIT)
    results = requests.get(url).json()
    
    result = {}                                #list that will store the results of all Json 
    result = results['response']['venues']     # filter specific location in results.json   
 
    dataframe = dataframe.append(result, ignore_index=True) # add filtered JSON data to empty dataframe.

    i += 1    # add 1 to i and restart loop
       
    if i == len(search_query):    # verify is i equals length of 'search_query'. If true, break loop.
        break

print('completed')

completed


In [45]:
# Clean: data is arranged to provide name, categorie titles, latitude and longitude.
# 'dataframe' is filtered with def to extract information from values.

filtered_columns = ['name', 'categories', 'location'] + [col for col in dataframe.columns if col.startswith('.location')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']      
    except:
        categories_list = row['venue.categories']        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
def get_lat_type(row):
    try:
        lat_list = row['location']      
    except:
        lat_list = row['venue.location']        
    if len(lat_list) == 0:
        return None
    else:
        return lat_list['lat']
    
def get_lng_type(row):
    try:
        lng_list = row['location']      
    except:
        lng_list = row['venue.location']        
    if len(lng_list) == 0:
        return None
    else:
        return lng_list['lng']
        
def get_city(row):
    try:
        city_list = row['location']      
    except:
        city_list = row['venue.location']        
    if len(city_list) == 0:
        return None
    else:
        return city_list['city']

In [46]:
# filter the category for each row

dataframe_filtered['name'] = dataframe_filtered['name'].str.replace(r'\.', '').str.replace(r'\$', '').str.replace(r'\'', '').str.replace(r'\&', 'and').str.replace(r'\?', '')
dataframe_filtered['name'] = dataframe_filtered['name'].str.replace(r'\$', '')
dataframe_filtered['name'] = dataframe_filtered['name'].str.replace(r'\'', '')


dataframe_filtered['name'].str.replace('.', '').astype(object)
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered['Latitude'] = dataframe_filtered.apply(get_lat_type, axis=1)
dataframe_filtered['Longitude'] = dataframe_filtered.apply(get_lng_type, axis=1)
dataframe_filtered['City'] = dataframe_filtered.apply(get_city, axis=1)
# Remove unwanted cities found within radius
indexNames = dataframe_filtered[ dataframe_filtered['City'] != 'Bronx' ].index
# Delete these row indexes from dataFrame
dataframe_filtered.drop(indexNames , inplace=True)
dataframe_filtered = dataframe_filtered[['name', 'categories', 'location', 'Latitude', 'Longitude', 'City', 'id']]

#### 4a. Visualize: Argentinian restaurant in Bronx County.

In [47]:
venues_map = folium.Map(location=[float(bronx_lat), float(bronx_lng)], zoom_start=12) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Bronx County
folium.features.CircleMarker(
    [float(bronx_lat), float(bronx_lng)],
    radius=10,
    color='red',
    popup='Bronx County',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.Latitude, dataframe_filtered.Longitude, dataframe_filtered.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

#### 4b. Kings County - Restaurant

In [48]:
########################################
### Retreive Four Square Information ###
########################################

#Kings County Results for 'Argentinean' locations
radius = 
# Kings lat/lng
kings_lat = float(df_NYCB_latlng['Kings County'].latitude)
kings_lng = float(df_NYCB_latlng['Kings County'].longitude)
lat = kings_lat = df_NYCB_latlng['Kings County']['latitude']
lng = kings_lng = df_NYCB_latlng['Kings County']['longitude']

In [49]:
# Create: Generate empty DataFrame to populate.
dataframe = pd.DataFrame()

In [50]:
# Data: Loop through a number of search queries. We call the Four Square API to search 
# and query specific keywords related to Argentinian restauraunts.
# We will use this loop for each county to find data by specific keywords. 
# 'dataframe' is populated on every loop and will provide results of query per location requested.

search_query = ['chimichurri', 'empanadas', 'Argentinian']
i = 0
while i < len(search_query):
    element = search_query[i]
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, element, radius, LIMIT)
    results = requests.get(url).json()
    
    result = {}                                #list that will store the results of all Json 
    result = results['response']['venues']     # filter specific location in results.json   
 
    dataframe = dataframe.append(result, ignore_index=True) # add filtered JSON data to empty dataframe.

    i += 1    # add 1 to i and restart loop
       
    if i == len(search_query):    # verify is i equals length of 'search_query'. If true, break loop.
        break

print('completed')

completed


In [51]:
# Clean: data is arranged to provide name, categorie titles, latitude and longitude.
# 'dataframe' is filtered with def to extract information from values.

filtered_columns = ['name', 'categories', 'location'] + [col for col in dataframe.columns if col.startswith('.location')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']      
    except:
        categories_list = row['venue.categories']        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
def get_lat_type(row):
    try:
        lat_list = row['location']      
    except:
        lat_list = row['venue.location']        
    if len(lat_list) == 0:
        return None
    else:
        return lat_list['lat']
    
def get_lng_type(row):
    try:
        lng_list = row['location']      
    except:
        lng_list = row['venue.location']        
    if len(lng_list) == 0:
        return None
    else:
        return lng_list['lng']
        
def get_city(row):
    try:
        city_list = row['location']      
    except:
        city_list = row['venue.location']        
    if len(city_list) == 0:
        return None
    else:
        return city_list['city']

In [52]:
# filter the category for each row

dataframe_filtered['name'] = dataframe_filtered['name'].str.replace(r'\.', '').str.replace(r'\$', '').str.replace(r'\'', '').str.replace(r'\&', 'and').str.replace(r'\?', '')
dataframe_filtered['name'] = dataframe_filtered['name'].str.replace(r'\$', '')
dataframe_filtered['name'] = dataframe_filtered['name'].str.replace(r'\'', '')


dataframe_filtered['name'].str.replace('.', '').astype(object)
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered['Latitude'] = dataframe_filtered.apply(get_lat_type, axis=1)
dataframe_filtered['Longitude'] = dataframe_filtered.apply(get_lng_type, axis=1)
dataframe_filtered['City'] = dataframe_filtered.apply(get_city, axis=1)
# Remove unwanted cities found within radius
indexNames = dataframe_filtered[ dataframe_filtered['City'] != 'Brooklyn' ].index
# Delete these row indexes from dataFrame
dataframe_filtered.drop(indexNames , inplace=True)
dataframe_filtered = dataframe_filtered[['name', 'categories', 'location', 'Latitude', 'Longitude', 'City', 'id']]

#### 4b. Visualize Argentinian restaurant in Kings County.

In [55]:
dataframe_filtered

,name,categories,location,Latitude,Longitude,City,id
0,Chimichurri Truck,Food Truck,"{'address': '4th Ave & 36th Street', 'lat': 40...",40.674334,-73.987591,Brooklyn,4aebcd53f964a520d2c421e3
1,Nuchas Artisan Empanadas,Food Stand,"{'address': '625 Atlantic Ave', 'crossStreet':...",40.683530,-73.975568,Brooklyn,5700348b498edac2f133c6ff
2,Cesars Empanadas,Food Truck,"{'address': 'Hanson Pl.', 'crossStreet': 'Ashl...",40.685412,-73.978311,Brooklyn,4dd4606fd4c0e6b33e3b6a01
3,Empanadas Doña Raquel,Food Truck,"{'lat': 40.68744850573533, 'lng': -73.94397565...",40.687449,-73.943976,Brooklyn,520d035e498eb1eec9d6f81d
4,El Coquí Empanadas,Empanada Restaurant,"{'address': '488A 7th Ave', 'crossStreet': 'Wi...",40.661922,-73.985775,Brooklyn,532f5fd9498e9223660cefd1
5,El Palacio De Las Empanadas Ecuatorianas,Food Truck,"{'lat': 40.70037662259608, 'lng': -73.93581276...",40.700377,-73.935813,Ridgewood,4f84b292e4b0d71c51e3e64e
6,Nuchas Artisan Empanadas,Empanada Restaurant,"{'lat': 40.69243190830672, 'lng': -73.98980634...",40.692432,-73.989806,New York,5ac3741323a2e62b779b71ec
7,Preparado Empanada Stand,Mexican Restaurant,"{'address': '5th Ave', 'crossStreet': '44th st...",40.648443,-74.006653,Brooklyn,4e909d050aafb4070c04bd0a


In [54]:
venues_map = folium.Map(location=[float(kings_lat), float(kings_lng)], zoom_start=12) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Bronx County
folium.features.CircleMarker(
    [float(kings_lat), float(kings_lng)],
    radius=10,
    color='red',
    popup='Kings County',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.Latitude, dataframe_filtered.Longitude, dataframe_filtered.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

#### 4c. New York County - Restaurant

In [66]:
########################################
### Retreive Four Square Information ###
########################################

#Kings County Results for 'Argentinean' locations
#radius = 
# Kings lat/lng
nyCounty_lat = float(df_NYCB_latlng['New York County'].latitude)
nyCounty_lng = float(df_NYCB_latlng['New York County'].longitude)
lat = nyCounty_lat = df_NYCB_latlng['New York County']['latitude']
lng = nyCounty_lng = df_NYCB_latlng['New York County']['longitude']

In [67]:
# Create: Generate empty DataFrame to populate.
dataframe = pd.DataFrame()

In [68]:
# Data: Loop through a number of search queries. We call the Four Square API to search 
# and query specific keywords related to Argentinian restauraunts.
# We will use this loop for each county to find data by specific keywords. 
# 'dataframe' is populated on every loop and will provide results of query per location requested.

search_query = ['chimichurri', 'empanadas', 'Argentinian']
i = 0
while i < len(search_query):
    element = search_query[i]
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, element, radius, LIMIT)
    results = requests.get(url).json()
    
    result = {}                                #list that will store the results of all Json 
    result = results['response']['venues']     # filter specific location in results.json   
 
    dataframe = dataframe.append(result, ignore_index=True) # add filtered JSON data to empty dataframe.

    i += 1    # add 1 to i and restart loop
       
    if i == len(search_query):    # verify is i equals length of 'search_query'. If true, break loop.
        break

print('completed')

completed


In [69]:
# Clean: data is arranged to provide name, categorie titles, latitude and longitude.
# 'dataframe' is filtered with def to extract information from values.

filtered_columns = ['name', 'categories', 'location'] + [col for col in dataframe.columns if col.startswith('.location')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']      
    except:
        categories_list = row['venue.categories']        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
def get_lat_type(row):
    try:
        lat_list = row['location']      
    except:
        lat_list = row['venue.location']        
    if len(lat_list) == 0:
        return None
    else:
        return lat_list['lat']
    
def get_lng_type(row):
    try:
        lng_list = row['location']      
    except:
        lng_list = row['venue.location']        
    if len(lng_list) == 0:
        return None
    else:
        return lng_list['lng']
        
def get_city(row):
    try:
        city_list = row['location']      
    except:
        city_list = row['venue.location']        
    if len(city_list) == 0:
        return None
    else:
        return city_list['city']

In [70]:
# filter the category for each row

dataframe_filtered['name'] = dataframe_filtered['name'].str.replace(r'\.', '').str.replace(r'\$', '').str.replace(r'\'', '').str.replace(r'\&', 'and').str.replace(r'\?', '')
dataframe_filtered['name'] = dataframe_filtered['name'].str.replace(r'\$', '')
dataframe_filtered['name'] = dataframe_filtered['name'].str.replace(r'\'', '')


dataframe_filtered['name'].str.replace('.', '').astype(object)
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered['Latitude'] = dataframe_filtered.apply(get_lat_type, axis=1)
dataframe_filtered['Longitude'] = dataframe_filtered.apply(get_lng_type, axis=1)
dataframe_filtered['City'] = dataframe_filtered.apply(get_city, axis=1)
# Remove unwanted cities found within radius
indexNames = dataframe_filtered[ dataframe_filtered['City'] != 'New York' ].index
# Delete these row indexes from dataFrame
dataframe_filtered.drop(indexNames , inplace=True)
dataframe_filtered = dataframe_filtered[['name', 'categories', 'location', 'Latitude', 'Longitude', 'City', 'id']]

#### 4c. Visualize Argentinian restaurant in Kings County.

In [71]:
dataframe_filtered

,name,categories,location,Latitude,Longitude,City,id
0,Chimichurri Grill,Steakhouse,"{'address': '133 E 61st St', 'lat': 40.763693,...",40.763693,-73.967563,New York,580d064b38fab1dc50148e0b
1,Chimichurri Grill,Steakhouse,"{'address': '609 9th Ave', 'crossStreet': '43r...",40.759229,-73.992032,New York,3fd66200f964a5208aea1ee3
4,La China Chimichurri,None,"{'address': '179th st', 'crossStreet': 'Amster...",40.844542,-73.933465,New York,4baf08b8f964a520fee73be3
5,El Primo Chimichurri,Food Truck,"{'address': '172nd and Amsterdam Ave', 'lat': ...",40.842469,-73.935434,New York,4d85a4de7e8ef04d229e25be
7,Empanadas Monumentales,Empanada Restaurant,"{'address': '3477 Broadway', 'crossStreet': '1...",40.824432,-73.952049,New York,55255b53498ebeabaf259e34
8,Empanadas Monumental,Empanada Restaurant,"{'address': '4093 Broadway', 'crossStreet': 'b...",40.844489,-73.938881,New York,4cbb75fabac93704fd46f27c
9,Empanadas and More,Empanada Restaurant,"{'address': '954 3rd Ave', 'lat': 40.760472, '...",40.760472,-73.967494,New York,5df90ee98ed5990008033c09
10,Empanadas,None,"{'lat': 40.754547, 'lng': -73.986351, 'labeled...",40.754547,-73.986351,New York,517ec4b6498ee0b8db6a4698
12,Empanadas Monumental,Spanish Restaurant,"{'address': '1344 Amsterdam Ave', 'lat': 40.81...",40.813807,-73.956254,New York,57fe5dc0498e9d3a7dfeefc6
13,Empanada Mama,Empanada Restaurant,"{'address': '765 9th Ave', 'crossStreet': 'btw...",40.764310,-73.988390,New York,4a453e4cf964a520f1a71fe3


In [72]:
venues_map = folium.Map(location=[float(nyCounty_lat), float(nyCounty_lng)], zoom_start=12) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Bronx County
folium.features.CircleMarker(
    [float(nyCounty_lat), float(nyCounty_lng)],
    radius=10,
    color='red',
    popup='New York County',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.Latitude, dataframe_filtered.Longitude, dataframe_filtered.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

#### 4d. Queens County - Restaurant

In [79]:
########################################
### Retreive Four Square Information ###
########################################

#Kings County Results for 'Argentinean' locations
#radius = 
# Kings lat/lng
queens_lat = float(df_NYCB_latlng['Queens County'].latitude)
queens_lng = float(df_NYCB_latlng['Queens County'].longitude)
lat = queens_lat = df_NYCB_latlng['Queens County']['latitude']
lng = queens_lng = df_NYCB_latlng['Queens County']['longitude']

In [80]:
# Create: Generate empty DataFrame to populate.
dataframe = pd.DataFrame()

In [81]:
# Data: Loop through a number of search queries. We call the Four Square API to search 
# and query specific keywords related to Argentinian restauraunts.
# We will use this loop for each county to find data by specific keywords. 
# 'dataframe' is populated on every loop and will provide results of query per location requested.

search_query = ['chimichurri', 'empanadas', 'Argentinian']
i = 0
while i < len(search_query):
    element = search_query[i]
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, element, radius, LIMIT)
    results = requests.get(url).json()
    
    result = {}                                #list that will store the results of all Json 
    result = results['response']['venues']     # filter specific location in results.json   
 
    dataframe = dataframe.append(result, ignore_index=True) # add filtered JSON data to empty dataframe.

    i += 1    # add 1 to i and restart loop
       
    if i == len(search_query):    # verify is i equals length of 'search_query'. If true, break loop.
        break

print('completed')

completed


In [82]:
# Clean: data is arranged to provide name, categorie titles, latitude and longitude.
# 'dataframe' is filtered with def to extract information from values.

filtered_columns = ['name', 'categories', 'location'] + [col for col in dataframe.columns if col.startswith('.location')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']      
    except:
        categories_list = row['venue.categories']        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
def get_lat_type(row):
    try:
        lat_list = row['location']      
    except:
        lat_list = row['venue.location']        
    if len(lat_list) == 0:
        return None
    else:
        return lat_list['lat']
    
def get_lng_type(row):
    try:
        lng_list = row['location']      
    except:
        lng_list = row['venue.location']        
    if len(lng_list) == 0:
        return None
    else:
        return lng_list['lng']
        
def get_city(row):
    try:
        city_list = row['location']      
    except:
        city_list = row['venue.location']        
    if len(city_list) == 0:
        return None
    else:
        return city_list['city']

In [83]:
# filter the category for each row

dataframe_filtered['name'] = dataframe_filtered['name'].str.replace(r'\.', '').str.replace(r'\$', '').str.replace(r'\'', '').str.replace(r'\&', 'and').str.replace(r'\?', '')
dataframe_filtered['name'] = dataframe_filtered['name'].str.replace(r'\$', '')
dataframe_filtered['name'] = dataframe_filtered['name'].str.replace(r'\'', '')


dataframe_filtered['name'].str.replace('.', '').astype(object)
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered['Latitude'] = dataframe_filtered.apply(get_lat_type, axis=1)
dataframe_filtered['Longitude'] = dataframe_filtered.apply(get_lng_type, axis=1)
dataframe_filtered['City'] = dataframe_filtered.apply(get_city, axis=1)
# Remove unwanted cities found within radius
#indexNames = dataframe_filtered[ dataframe_filtered['City'] != 'New York' ].index
# Delete these row indexes from dataFrame
#dataframe_filtered.drop(indexNames , inplace=True)
dataframe_filtered = dataframe_filtered[['name', 'categories', 'location', 'Latitude', 'Longitude', 'City', 'id']]

#### 4d. Visualize Argentinian restaurant in Queens County.

In [84]:
dataframe_filtered

,name,categories,location,Latitude,Longitude,City,id
0,Chimichurri Charcoal Kitchen,Portuguese Restaurant,"{'address': '450 Rockaway Tpke', 'crossStreet'...",40.627598,-73.736664,Lawrence,56801b7e498e9b9cf26e2fd4
1,Palacio del chimichurri,None,"{'address': '108-09 101st Ave', 'lat': 40.6873...",40.687341,-73.835174,Richmond Hill,4cf984f2ee9cb60c869d84ad
2,Empanadas and Salchipapas,Food Truck,{'address': 'Front of Atlantic Mall/Terminal E...,40.677622,-73.864284,Brooklyn,4d2ba35e915fa0937fc7180a
3,Viva Mis Empanadas,Mexican Restaurant,"{'address': '133-11 Crossbay Blvd', 'lat': 40....",40.675006,-73.842869,Ozone Park,56e49e1f498efc6a65da47a3
4,Ricas Empanadas,Empanada Restaurant,"{'address': '159-35 Crossbay Blvd', 'lat': 40....",40.659229,-73.839867,Ozone Park,54e7c833498ebe5df6a964ce
5,Mamas Empanadas,Empanada Restaurant,"{'address': '91-11 Jamaica Ave', 'lat': 40.693...",40.693222,-73.853380,Woodhaven,5b92f318911fc4002ced9fa5


In [85]:
venues_map = folium.Map(location=[float(queens_lat), float(queens_lng)], zoom_start=12) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Bronx County
folium.features.CircleMarker(
    [float(queens_lat), float(queens_lng)],
    radius=10,
    color='red',
    popup='Queens County',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.Latitude, dataframe_filtered.Longitude, dataframe_filtered.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

#### 4e. Richmond County - Restaurant

In [98]:
########################################
### Retreive Four Square Information ###
########################################

#Kings County Results for 'Argentinean' locations
#radius = 
# Kings lat/lng
richmond_lat = float(df_NYCB_latlng['Richmond County'].latitude)
richmond_lng = float(df_NYCB_latlng['Richmond County'].longitude)
lat = richmond_lat = df_NYCB_latlng['Richmond County']['latitude']
lng = richmond_lng = df_NYCB_latlng['Richmond County']['longitude']

In [99]:
# Create: Generate empty DataFrame to populate.
dataframe = pd.DataFrame()

In [100]:
# Data: Loop through a number of search queries. We call the Four Square API to search 
# and query specific keywords related to Argentinian restauraunts.
# We will use this loop for each county to find data by specific keywords. 
# 'dataframe' is populated on every loop and will provide results of query per location requested.

search_query = ['chimichurri', 'empanadas', 'Argentinian']
i = 0
while i < len(search_query):
    element = search_query[i]
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, element, radius, LIMIT)
    results = requests.get(url).json()
    
    result = {}                                #list that will store the results of all Json 
    result = results['response']['venues']     # filter specific location in results.json   
 
    dataframe = dataframe.append(result, ignore_index=True) # add filtered JSON data to empty dataframe.

    i += 1    # add 1 to i and restart loop
       
    if i == len(search_query):    # verify is i equals length of 'search_query'. If true, break loop.
        break

print('completed')

completed


In [102]:
dataframe.index

RangeIndex(start=0, stop=0, step=1)

In [95]:
# Clean: data is arranged to provide name, categorie titles, latitude and longitude.
# 'dataframe' is filtered with def to extract information from values.
if d in dataframe
filtered_columns = ['name', 'categories', 'location'] + [col for col in dataframe.columns if col.startswith('.location')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']      
    except:
        categories_list = row['venue.categories']        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
def get_lat_type(row):
    try:
        lat_list = row['location']      
    except:
        lat_list = row['venue.location']        
    if len(lat_list) == 0:
        return None
    else:
        return lat_list['lat']
    
def get_lng_type(row):
    try:
        lng_list = row['location']      
    except:
        lng_list = row['venue.location']        
    if len(lng_list) == 0:
        return None
    else:
        return lng_list['lng']
        
def get_city(row):
    try:
        city_list = row['location']      
    except:
        city_list = row['venue.location']        
    if len(city_list) == 0:
        return None
    else:
        return city_list['city']

KeyError: "None of [Index(['name', 'categories', 'location', 'id'], dtype='object')] are in the [columns]"

In [83]:
# filter the category for each row

dataframe_filtered['name'] = dataframe_filtered['name'].str.replace(r'\.', '').str.replace(r'\$', '').str.replace(r'\'', '').str.replace(r'\&', 'and').str.replace(r'\?', '')
dataframe_filtered['name'] = dataframe_filtered['name'].str.replace(r'\$', '')
dataframe_filtered['name'] = dataframe_filtered['name'].str.replace(r'\'', '')


dataframe_filtered['name'].str.replace('.', '').astype(object)
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered['Latitude'] = dataframe_filtered.apply(get_lat_type, axis=1)
dataframe_filtered['Longitude'] = dataframe_filtered.apply(get_lng_type, axis=1)
dataframe_filtered['City'] = dataframe_filtered.apply(get_city, axis=1)
# Remove unwanted cities found within radius
#indexNames = dataframe_filtered[ dataframe_filtered['City'] != 'New York' ].index
# Delete these row indexes from dataFrame
#dataframe_filtered.drop(indexNames , inplace=True)
dataframe_filtered = dataframe_filtered[['name', 'categories', 'location', 'Latitude', 'Longitude', 'City', 'id']]

#### 4d. Visualize Argentinian restaurant in Queens County.

In [84]:
dataframe_filtered

,name,categories,location,Latitude,Longitude,City,id
0,Chimichurri Charcoal Kitchen,Portuguese Restaurant,"{'address': '450 Rockaway Tpke', 'crossStreet'...",40.627598,-73.736664,Lawrence,56801b7e498e9b9cf26e2fd4
1,Palacio del chimichurri,None,"{'address': '108-09 101st Ave', 'lat': 40.6873...",40.687341,-73.835174,Richmond Hill,4cf984f2ee9cb60c869d84ad
2,Empanadas and Salchipapas,Food Truck,{'address': 'Front of Atlantic Mall/Terminal E...,40.677622,-73.864284,Brooklyn,4d2ba35e915fa0937fc7180a
3,Viva Mis Empanadas,Mexican Restaurant,"{'address': '133-11 Crossbay Blvd', 'lat': 40....",40.675006,-73.842869,Ozone Park,56e49e1f498efc6a65da47a3
4,Ricas Empanadas,Empanada Restaurant,"{'address': '159-35 Crossbay Blvd', 'lat': 40....",40.659229,-73.839867,Ozone Park,54e7c833498ebe5df6a964ce
5,Mamas Empanadas,Empanada Restaurant,"{'address': '91-11 Jamaica Ave', 'lat': 40.693...",40.693222,-73.853380,Woodhaven,5b92f318911fc4002ced9fa5


In [85]:
venues_map = folium.Map(location=[float(queens_lat), float(queens_lng)], zoom_start=12) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Bronx County
folium.features.CircleMarker(
    [float(queens_lat), float(queens_lng)],
    radius=10,
    color='red',
    popup='Queens County',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.Latitude, dataframe_filtered.Longitude, dataframe_filtered.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<a id="results"></a>

### Results & Discussion

<a id="conclusion"></a>

### Conclusion